<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [1]:
!gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
!unzip archive.zip

Downloading...
From: https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph
To: /content/archive.zip
84.9MB [00:00, 87.8MB/s]
Archive:  archive.zip
  inflating: training.1600000.processed.noemoticon.csv  


Импортируем нужные библиотеки:

In [2]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [3]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

# device = "cuda" if torch.cuda.is_available() else "cpu"

device = "cpu"

In [4]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", 
                   encoding="latin", 
                   header=None, 
                   names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [ ]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
data.shape

(1600000, 6)

Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [ ]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Как видим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [5]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8) # округление вверх

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [6]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

import re

nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
dev_data["text"][0]

'@chrishasboobs AHHH I HOPE YOUR OK!!! '

Токенизация с nltk.WordPunctTokenizer()

In [7]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ chrishasboobs ahhh i hope your ok !!!


In [8]:
# string.punctuation = !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~

filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

chrishasboobs ahhh hope your


Токенизация через регулярные выражения

In [9]:
line = re.findall('[a-zA-Z]+', dev_data["text"][0].lower())
print(" ".join(line))

chrishasboobs ahhh i hope your ok


In [10]:
wnl = nltk.WordNetLemmatizer()

filtered_line = [wnl.lemmatize(word) for word in line if word not in stopWords]
filtered_line

['chrishasboobs', 'ahhh', 'hope', 'ok']

Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [11]:
word2vec = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(emb_line).shape)

(300,)


Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

In [ ]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line \
                 if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [ ]:
from torch.utils.data import Dataset, random_split

class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, 
                 target_column: str, word2vec: gensim.models.Word2Vec):
        # self.tokenizer = re
        self.tokenizer = nltk.WordPunctTokenizer()
        # self.lemmatizer = nltk.WordNetLemmatizer()
        # self.stopwords = set(stopwords.words('english'))
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):

        # Получи все токены из текста и профильтруй их
        line = self.tokenizer.tokenize(text.lower())
        # line = self.tokenizer.findall('[a-zA-Z]+', text.lower())

        # оставила слова в 3 буквы, т.к. они распространены в английском
        tokens = [w for w in line if all(c not in string.punctuation for c in w)]
        tokens = [w for w in tokens if len(w) > 2] 
        # tokens = [w for w in tokens if w not in self.stopwords and len(w) > 2] 
        # tokens = [self.lemmatizer.lemmatize(w) for w in tokens] 
 
        return tokens

    def get_embeddings_(self, tokens):
        # Получи эмбеддинги слов и усредни их
        embeddings = [(self.word2vec.get_vector(w) - self.mean) / self.std \
                      for w in tokens if w in self.word2vec]                                                                               
                                                                                                        
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предложении (которые остались после токенизации и удаления коротких слов, конечно). 

In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
# Взяли все индексы (:) и поделили их число на 1000 (:1000), осталось 1280 вместо 1280000
example_indexes = indexes[::1000] 

examples = {"features": [np.mean(dev[i]["feature"], axis=0) for i in example_indexes],     
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
# Обучи PCA на эмбеддингах слов
examples["transformed_features"] = pca.fit_transform(examples["features"]) 

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1003', ...)

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднение векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [ ]:
from torch.utils.data import DataLoader

batch_size = 1024
num_workers = 2

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    # return {"features": torch.FloatTensor(features), 
    #         "targets": torch.LongTensor(targets)}
    # заменила Long на Float, иначе ошибка при обучении
    return {"features": torch.FloatTensor(features), 
            "targets": torch.FloatTensor(targets)} 

train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers,
                          shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, 
                          shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
len(train)

1024000

Определим функции для тренировки и теста модели:

In [ ]:
from tqdm.notebook import tqdm

def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):

    # e - это из "for e in range(num_epochs):", см. код дальше
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}") 
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)
        targets = targets.reshape(-1, 1)

        # set parameter gradients to zero
        optimizer.zero_grad()

        # Получи предсказания модели
        Y_pred = model(features)
        loss = criterion(Y_pred, targets) # Посчитай лосс
        loss.backward()
        optimizer.step()  # Обнови параметры модели

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"): 
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)
            targets = targets.reshape(-1, 1)

            # Получи предсказания модели
            Y_hat = model(features)
            loss = criterion(Y_hat, targets) # Посчитай лосс

            Y_hat = torch.sigmoid(Y_hat)
            Y_hat = (Y_hat>0.5).type(torch.long)
            
            # Посчитай точность модели
            acc = torch.sum(Y_hat == targets) / len(targets)

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, \
                           Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), 
            "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


In [ ]:
import torch.nn as nn
from torch.optim import Adam

# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
lr = 1e-2
num_epochs = 1

# model = nn.Sequential(
#   nn.Linear(vector_size, 128),
#   nn.ReLU(),
#   nn.Linear(128, 64),
#   nn.ReLU(),
#   nn.Linear(64, 32),
#   nn.ReLU(),
#   nn.Linear(32, 1)
# )

model = nn.Sequential(
  nn.Linear(vector_size, 128),
  nn.ReLU(),
  nn.Linear(128, 1))

# model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=lr)# Твой оптимайзер

Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

# НС в 4 слоя

# {'Test Loss': 0.5086260961294174, 'Test Acc': 0.74740234375}    nltk.WordPunctTokenizer() + токены от 4 букв
# {'Test Loss': 0.4792954614162445, 'Test Acc': 0.76518359375}    nltk.WordPunctTokenizer() + токены от 3 букв 
# {'Test Loss': 0.5027369836568832, 'Test Acc': 0.75190625}       re - слова и цифры + лемматизация
# {'Test Loss': 0.5026048966646195, 'Test Acc': 0.7523671875}     re - только слова + лемматизация
# {'Test Loss': 0.5023029655218124, 'Test Acc': 0.75173828125}    nltk.WordPunctTokenizer() + токены от 3 букв + лемматизация + стопслова
# {'Test Loss': 0.5050759416818619, 'Test Acc': 0.74971484375}    nltk.WordPunctTokenizer() + токены от 3 букв + стопслова

# НС в 2 слоя
# {'Test Loss': 0.48422147274017335, 'Test Acc': 0.76534375}      nltk.WordPunctTokenizer() + токены от 3 букв 


{'Test Loss': 0.48422147274017335, 'Test Acc': 0.76534375}


In [ ]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

# НС в 4 слоя

# {'Test Loss': 0.5124567241523974, 'Test Acc': 0.7442467052715654}   nltk.WordPunctTokenizer() + токены от 4 букв
# {'Test Loss': 0.4773869004112463, 'Test Acc': 0.7667887629792333}   nltk.WordPunctTokenizer() + токены от 3 букв  
# {'Test Loss': 0.5021114643579855, 'Test Acc': 0.753113767971246}    re - слова и цифры + лемматизация
# {'Test Loss': 0.5022929027057684, 'Test Acc': 0.7525116064297125}   re - только слова + лемматизация
# {'Test Loss': 0.501444010879285, 'Test Acc': 0.7527331269968051}    nltk.WordPunctTokenizer() + токены от 3 букв + лемматизация

# НС в 2 слоя

# {'Test Loss': 0.4827558478227439, 'Test Acc': 0.7667450828674122}   nltk.WordPunctTokenizer() + токены от 3 букв


{'Test Loss': 0.4827558478227439, 'Test Acc': 0.7667450828674122}


Токенайзер nltk.WordPunctTokenizer() + токены от 3 букв выигрывает по качеству, дальше будем использовать его. Также нейронная сеть в 2 слоя показала примерно то же качество, что и сеть в 4 слоя, значит нет необходимости в таком количестве слоев, достаточно 2-х. Но проверим это предположение на более сложных задачах дальше.

## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

1. Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)

# Без весов на дальность слова

In [ ]:
from torch.utils.data import Dataset, random_split

class TwitterDataset_Context(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, 
                 target_column: str, word2vec: gensim.models.Word2Vec, window: int):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec
        self.window =  window

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        line = self.tokenizer.tokenize(text.lower())
        tokens = [w for w in line if all(c not in string.punctuation for c in w)]
        tokens = [w for w in tokens if len(w) > 2] 
        return tokens

    def get_embeddings_(self, tokens):
        # пустой шаблон - число токенов в твите на 300(W2V)
        embeddings = np.zeros((len(tokens), self.word2vec.vector_size))  

        for i in range(len(tokens)):

          if tokens[i] in self.word2vec:
              # Получи эмбеддинги слов и усредни их
              embeddings[i] = (self.word2vec.get_vector(tokens[i]) - self.mean) \
                               / self.std 

          else:
            # определяем левую границу контекста
            left = max(i - self.window, 0)
            if i == 0:
              left = 1

            # определяем правую границу контекста
            right = min(i + self.window + 1, len(tokens))
            
            num = 0
            for e in range(left, right):
                if tokens[e] in self.word2vec:
                   embeddings[i] += (self.word2vec.get_vector(tokens[e]) - self.mean) \
                                    / self.std
                   num += 1 
                   # tokens[i], если i попадает в  range(left, right) все равно
                   # не будет учитываться в num, т.к. его нет в self.word2vec
            if num != 0:      
              # усредняем вектор неизвестного слова по числу известных слов из контекста   
              embeddings[i] = embeddings[i] / num   

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
          embeddings = np.array(embeddings)
          if len(embeddings.shape) == 1:
              embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev_Context = TwitterDataset_Context(dev_data, "text", "emotion", word2vec, 3)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 1024
num_workers = 2

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.FloatTensor(targets)}

train_size = math.ceil(len(dev_Context) * 0.8)

train, valid = random_split(dev_Context, [train_size, len(dev_Context) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, 
                          shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, 
                          shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
from tqdm.notebook import tqdm

def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}") 
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)
        targets = targets.reshape(-1, 1)

        # set parameter gradients to zero
        optimizer.zero_grad()

        # Получи предсказания модели
        Y_pred = model(features)
        loss = criterion(Y_pred, targets) # Посчитай лосс
        loss.backward()
        optimizer.step()  # Обнови параметры модели

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")

def testing(model, criterion, test_loader, device="cpu"): 
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)
            targets = targets.reshape(-1, 1)

            # Получи предсказания модели
            Y_hat = model(features)
            loss = criterion(Y_hat, targets) # Посчитай лосс

            Y_hat = torch.sigmoid(Y_hat)
            Y_hat = (Y_hat>0.5).type(torch.long)
            
            # Посчитай точность модели
            acc = torch.sum(Y_hat == targets) / len(targets)

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, \
                           Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), 
            "Test Acc": mean_acc / len(test_loader)}

In [ ]:
import torch.nn as nn
from torch.optim import Adam

# Не забудь поиграться с параметрами ;)
vector_size = dev_Context.word2vec.vector_size
lr = 1e-2
num_epochs = 1

model = nn.Sequential( # Твоя модель
  nn.Linear(vector_size, 128),
  nn.ReLU(),
  nn.Linear(128, 64),
  nn.ReLU(),
  nn.Linear(64, 32),
  nn.ReLU(),
  nn.Linear(32, 1)
)

# model = nn.Sequential(
#   nn.Linear(vector_size, 128),
#   nn.ReLU(),
#   nn.Linear(128, 1))

model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=lr)# Твой оптимайзер

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

# НС в 4 слоя
# {'Test Loss': 0.47848300874233246, 'Test Acc': 0.76915625}

# НС в 2 слоя

# {'Test Loss': 0.49272973656654356, 'Test Acc': 0.7605}
# с усреднением вектора неизвестного слова по числу известных слов из контекста,
#  nltk.WordPunctTokenizer() + токены от 3 букв 


{'Test Loss': 0.47848300874233246, 'Test Acc': 0.76915625}


In [ ]:
test_loader = DataLoader(
    TwitterDataset_Context(test_data, "text", "emotion", word2vec, 3), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

# НС в 4 слоя
# {'Test Loss': 0.47829622068344213, 'Test Acc': 0.7686233276757188}

# НС в 2 слоя

# {'Test Loss': 0.4916354916727962, 'Test Acc': 0.7613849091453674}
# с усреднением вектора неизвестного слова по числу известных слов из контекста,
# nltk.WordPunctTokenizer() + токены от 3 букв 


{'Test Loss': 0.47829622068344213, 'Test Acc': 0.7686233276757188}


При использовании эмбеддингов незнакомых слов как сумму эмбеддингов всех слов из их контекста (без весов на дальность слова), на тесте получаем результат 0.7686 при 4-х слойной НС - также, как когда мы не учитываем незнакомые для word2vec слова. Там результат был - 0.7667. 
В этой задаче 4-х слойная сеть работает чуть лучше, чем 2-х слойная.

# С весами на дальность слова

In [ ]:
from torch.utils.data import Dataset, random_split

class TwitterDataset_Context_Weigts(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str,
                 target_column: str, word2vec: gensim.models.Word2Vec, window: int):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec
        self.window =  window

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        line = self.tokenizer.tokenize(text.lower())
        tokens = [w for w in line if all(c not in string.punctuation for c in w)]
        tokens = [w for w in tokens if len(w) > 2] 
        return tokens

    def get_embeddings_(self, tokens):

        embeddings = np.zeros((len(tokens), self.word2vec.vector_size))    

        for i in range(len(tokens)):

          if tokens[i] in self.word2vec:
              embeddings[i] = (self.word2vec.get_vector(tokens[i]) - self.mean) \
                              / self.std # Получи эмбеддинги слов и усредни их

          else:
            # определяем левую границу контекста
            left = max(i - self.window, 0)
            if i == 0:
              left = 1

            # определяем правую границу контекста
            right = min(i + self.window + 1, len(tokens))
            
            num = 0
            for e in range(left, right):

                if tokens[e] in self.word2vec: 
                   num += 1

                   if abs(i-e) == 1:
                    embeddings[i] += ((self.word2vec.get_vector(tokens[e]) - \
                                     self.mean) / self.std) * 1.5
                   elif abs(i-e) == 2:
                    embeddings[i] += ((self.word2vec.get_vector(tokens[e]) - \
                                     self.mean) / self.std) 
                   else: 
                    embeddings[i] +=((self.word2vec.get_vector(tokens[e]) - \
                                     self.mean) / self.std) * 0.5

                  #  if abs(i-e) == 1:
                  #   embeddings[i] += ((self.word2vec.get_vector(tokens[e]) - \
                  #                      self.mean) / self.std) * 0.3
                  #  elif abs(i-e) == 2:
                  #   embeddings[i] += ((self.word2vec.get_vector(tokens[e]) - \
                  #                      self.mean) / self.std) * 0.2
                  #  else: 
                  #   embeddings[i] +=((self.word2vec.get_vector(tokens[e]) - \
                  #                     self.mean) / self.std) * 0.1

  # Я взяла коэффициенты, которые в сумме дают 6 (при наличии всех слов в контексте): (1.5+1+0.5)*2 = 6 (коэф.с двух сторон),
  # чтобы при делении потом на num получить вектор, имеющий смысл в этом пространстве
  # Но скорее всего в контексте не будет всех 6 слов, тогда такой выбор имеет мало смысла...
  # Или алгоритм подбора коэффициентов иной, например, в чате говорили про коэф., которые в сумме дают 1: (0.3 + 0.2 + 0.1)*2 = 1 ?
            
                if num != 0:       
                  embeddings[i] = embeddings[i] / num # усредняем вектор неизвестного слова по числу известных слов из контекста      

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
          embeddings = np.array(embeddings)
          if len(embeddings.shape) == 1:
              embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev_Context = TwitterDataset_Context_Weigts(dev_data, "text", "emotion", word2vec, 3)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 1024
num_workers = 2

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]
    
    return {"features": torch.FloatTensor(features), "targets": torch.FloatTensor(targets)}

train_size = math.ceil(len(dev_Context) * 0.8)

train, valid = random_split(dev_Context, [train_size, len(dev_Context) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers,
                          shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers,
                          shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
import torch.nn as nn
from torch.optim import Adam

vector_size = dev_Context.word2vec.vector_size
lr = 1e-2
num_epochs = 1

model = nn.Sequential( # Твоя модель
  nn.Linear(vector_size, 128),
  nn.ReLU(),
  nn.Linear(128, 64),
  nn.ReLU(),
  nn.Linear(64, 32),
  nn.ReLU(),
  nn.Linear(32, 1)
)

# model = nn.Sequential(
#   nn.Linear(vector_size, 128),
#   nn.ReLU(),
#   nn.Linear(128, 1))

model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=lr)# Твой оптимайзер

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

# НС в 4 слоя

# {'Test Loss': 0.48236547696590426, 'Test Acc': 0.76722265625} nltk.WordPunctTokenizer() + токены от 3 букв, коэф. 1.5, 1, 0.5
# {'Test Loss': 0.48037377166748046, 'Test Acc': 0.7666015625}  nltk.WordPunctTokenizer() + токены от 3 букв, коэф. 0.3, 0.2, 0.1

# НС в 2 слоя

# {'Test Loss': 0.48998295903205874, 'Test Acc': 0.7587109375} nltk.WordPunctTokenizer() + токены от 3 букв, коэф. 0.3, 0.2, 0.1


{'Test Loss': 0.48236547696590426, 'Test Acc': 0.76722265625}


In [ ]:
test_loader = DataLoader(
    TwitterDataset_Context_Weigts(test_data, "text", "emotion", word2vec, 3), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

# НС в 4 слоя

# {'Test Loss': 0.48219194294164736, 'Test Acc': 0.7666046825079872} nltk.WordPunctTokenizer() + токены от 3 букв, коэф. 1.5, 1, 0.5
# {'Test Loss': 0.4797307040554266, 'Test Acc': 0.7686514077476039} nltk.WordPunctTokenizer() + токены от 3 букв, коэф. 0.3, 0.2, 0.1

# НС в 2 слоя

# {'Test Loss': 0.4898856387922939, 'Test Acc': 0.7600713857827476}


{'Test Loss': 0.48219194294164736, 'Test Acc': 0.7666046825079872}


Добавление весов на дальность слова в контексте с коэффициентами  0.3, 0.2, 0.1 снова почти не изменило качество по сравнению с тем, когда мы не учитываем незнакомые для word2vec слова (0.7686 стало и 0.7667 было)
Тут также 4-х слойная сеть работает чуть лучше, чем 2-х слойная.

2. Для каждого слова текста получим его эмбеддинг из Tfidf с помощью TfidfVectorizer из sklearn. Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

In [ ]:
# Идеи из чата DLS:

# Я бы завел отдельный токен <UNK> и заменил все неизвестные слова не него. 
# Для него посчитал бы TFiDF и сказал бы, что это эмбединг нашего слова. 

# ну да. Подкину вам идею. Можно попробовать обучить маленький word2vec только для одного токена <UNK>. 
# То есть смотрите с кем встречается этот токен и еще сэмплируете с кем он не встречается (negative sampling). 
# Вектора остальных слов не трогаете. Просто градиенты считаете только для одного вектора — эмбединга <UNK>. 
# Сходу вроде норм идея, но может не сработать. В общем если хотите похимичить, то после выполнения основного задания можно попробовать.

# Но можно еще перевзвешивать вес эмбединга данного слова с помощью tf idf
# Ну то есть по умолчанию мы берем среднее
# А так можно брать взвешенную сумму, где вес слова это его tf idf
# Можно просто сделать tf idf всего корпуса, а затем понизить его размерность до 300 через всякие PCA

# Я попробовал такой вариант. Использовал IDF слова из tfidf в качестве веса для вектора из word2vec. 
# Результат score получился абсолютно идентичен чистому word2vec.

# Перевзвешивать можно, и это имеет смысл, выделяем значимые слова, а вес случайно добавленных в среднее слов -типа артикли- уменьшаем.
#  Можно просто этот вес в хвост вектора word2vec добавить. Жаль проблему слов, которых нет в word2vec это не решает.

# Ну и к лучшему, ведь word2vec хоть какое-то геометрическое обоснование имеет. 
# А если к word2vec векторам потом какие-то совершенно другие добавлять? 
# Ну испортим значения в каждой размерности. Что ещё может быть? С чего бы это улучшило результат классификации?

# Конкатенация - Это разумнее. То есть у некоторых слов одна часть вектора нулевая была (слова нет в word2vec), а хвост какой-то другой? И это помогает.
# да, именно так, для тех, что нет в эмбеддинге - будет ненулевая добавка

Я выбрала идею с конкатенацией вектора tfidf в конец вектора w2v 

Сначала сделаем датасет, в котором не эмбединги слов, а строки из токенов - для обучения tfidf



In [12]:
from torch.utils.data import Dataset, random_split


class TwitterDataset_TFIDF(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.label2num = lambda label: 0 if label == 0 else 1

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)

        return {"feature": tokens, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их

        line = self.tokenizer.tokenize(text.lower())
        tokens = [w for w in line if all(c not in string.punctuation for c in w)]
        tokens = [w for w in tokens if len(w) > 2] 
        tokens = ' '.join(tokens)
        return tokens

    def __len__(self):
        return self.data.shape[0]

In [13]:
dev_TFIDF = TwitterDataset_TFIDF(dev_data, "text", "emotion")

In [ ]:
dev_TFIDF[0]["feature"]

'chrishasboobs ahhh hope your'

In [14]:
# выделим train часть для обучения tfidf
# и зафиксируем генератор, чтобы позже также поделить данные для обучения НС

train_size = math.ceil(len(dev_TFIDF) * 0.8)
train_TFIDF, _ = random_split(dev_TFIDF, 
                        [train_size, len(dev_TFIDF) - train_size], 
                        generator=torch.Generator().manual_seed(42))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# from sklearn.decomposition import TruncatedSVD - НЕ ХВАТАЕТ ПАМЯТИ, ЧТОБЫ РАЗЛОЖИТЬ ВСЕ ДАННЫЕ

In [16]:
# создаем словарь с токенами, 
# т.к. из dev_TFIDF токены доставать можно только через индекс dev_TFIDF[0]["feature"]
# можно сделать это проще?

indexes = np.arange(len(train_TFIDF))

examples = {"features": [(train_TFIDF[i]["feature"]) for i in indexes]}  
print(len(examples["features"]))

1024000


In [17]:
# vectorizer = TfidfVectorizer() # max_df=0.7-0.9 не меняет количество слов 
vectorizer = TfidfVectorizer(min_df=3)
vectorizer.fit(examples["features"]) 
vectors = vectorizer.transform(examples["features"]) 

In [18]:
# транспонируем из (твиты на слова) в (слова на твиты) 
# для удобства дальнейшего svd разложения

vectors_T = vectors.T 
vectors_T.shape

# (479782, 1024000) TfidfVectorizer()
# (92941, 1024000) TfidfVectorizer(min_df=3)
# (54800, 1024000) TfidfVectorizer(min_df=5)

(92941, 1024000)

In [19]:
from scipy.sparse.linalg import svds

In [20]:
U_100, *_ = svds(vectors_T, k = 100)
vectors_T.shape, U_100.shape # размерность - число слов на 100

((92941, 1024000), (92941, 100))

In [21]:
# Нормализуем U_100

mean_tfidf = np.mean(U_100, 0)
std_tfidf = np.std(U_100, 0)
U_100_norm = (U_100 - mean_tfidf) / std_tfidf

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
# from numpy import savetxt

# savetxt('/gdrive/MyDrive/U_100_norm.csv', U_100_norm, delimiter=';')

In [ ]:
# from numpy import genfromtxt

# U_100_norm = genfromtxt('/gdrive/MyDrive/U_100_norm.csv', delimiter=';')

In [22]:
# Создадим словарь {токен: его нормализованный вектор tfidf размерности 100}

tfidf_dict = dict(zip(vectorizer.get_feature_names(), U_100_norm))

In [ ]:
# tfidf_dict['aarondior']

In [23]:
from torch.utils.data import Dataset, random_split

class TwitterDataset_TFIDF_plus_W2V(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, 
                 target_column: str, word2vec: gensim.models.Word2Vec, tfidf_dict):
        self.tokenizer = nltk.WordPunctTokenizer()

        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec
        self.tfidf_dict = tfidf_dict

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        line = self.tokenizer.tokenize(text.lower())
        tokens = [w for w in line if all(c not in string.punctuation for c in w)]
        tokens = [w for w in tokens if len(w) > 2] 
        return tokens

    def get_embeddings_(self, tokens):
        
        # пустой шаблон - число токенов в твите на 400
        embeddings = np.zeros((len(tokens), self.word2vec.vector_size + 100))    

        for i in range(len(tokens)):

          if tokens[i] in self.word2vec:
              embeddings_w2v = (self.word2vec.get_vector(tokens[i]) - self.mean) / self.std 
          else:    
              embeddings_w2v = np.zeros(self.word2vec.vector_size)

          if tokens[i] in self.tfidf_dict.keys():
              # Вектор матрицы TFIDF, соответствующий токену 
              embeddings_tfidf = self.tfidf_dict[tokens[i]]
          else:
              embeddings_tfidf = np.zeros(100)

          embeddings[i] += np.concatenate((embeddings_w2v, embeddings_tfidf), axis=None)

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size + 100))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [24]:
dev_TFIDF_plus_W2V = TwitterDataset_TFIDF_plus_W2V(dev_data, "text", "emotion", word2vec, tfidf_dict) 

In [ ]:
# dev[0]["feature"]

In [25]:
from torch.utils.data import DataLoader

batch_size = 1024
num_workers = 2

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.FloatTensor(targets)}

# train_size = math.ceil(len(dev_TFIDF_plus_W2V) * 0.8) определили выше

train, valid = random_split(dev_TFIDF_plus_W2V, 
                            [train_size, len(dev_TFIDF_plus_W2V) - train_size], 
                            generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers,
                          shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers,
                          shuffle=False, drop_last=False, collate_fn=average_emb)

In [34]:
import torch.nn as nn
from torch.optim import Adam

vector_size = dev_TFIDF_plus_W2V.word2vec.vector_size + 100
lr = 1e-2
num_epochs = 1

# model = nn.Sequential( # Твоя модель
#   nn.Linear(vector_size, 256),
#   nn.ReLU(),
#   nn.Linear(256, 128),
#   nn.ReLU(),
#   nn.Linear(128, 64),
#   nn.ReLU(),
#   nn.Linear(64, 1)
# )

model = nn.Sequential(
  nn.Linear(vector_size, 128),
  nn.ReLU(),
  nn.Linear(128, 1))

model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=lr)# Твой оптимайзер

In [35]:
from tqdm.notebook import tqdm

def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}") # e - это из "for e in range(num_epochs):", см. код дальше
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)
        targets = targets.reshape(-1, 1)

        # set parameter gradients to zero
        optimizer.zero_grad()

        # Получи предсказания модели
        Y_pred = model(features)
        loss = criterion(Y_pred, targets) # Посчитай лосс
        loss.backward()
        optimizer.step()  # Обнови параметры модели

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"): 
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)
            targets = targets.reshape(-1, 1)

            # Получи предсказания модели
            Y_hat = model(features)
            loss = criterion(Y_hat, targets) # Посчитай лосс

            Y_hat = torch.sigmoid(Y_hat)
            Y_hat = (Y_hat>0.5).type(torch.long)
            
            # Посчитай точность модели
            acc = torch.sum(Y_hat == targets) / len(targets)

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

In [36]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

# НС 4 слоя

# {'Test Loss': 0.47089520156383513, 'Test Acc': 0.7716171875} TfidfVectorizer(min_df=3)

# НС 2 слоя

# {'Test Loss': 0.480280495762825, 'Test Acc': 0.7677890625} TfidfVectorizer(min_df=3)

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.480280495762825, 'Test Acc': 0.7677890625}


In [29]:
# проверка на тестовом датасете
# для этого получим tf-idf матрицу для тестовых данных

test_TFIDF = TwitterDataset_TFIDF(test_data, "text", "emotion")
test_indexes = np.arange(len(test_TFIDF))
test_examples = {"features": [(test_TFIDF[i]["feature"]) for i in test_indexes]}  
test_vectors = vectorizer.transform(test_examples["features"]) 
test_vectors_T = test_vectors.T 
test_vectors_T.shape

(92941, 320000)

In [30]:
test_U_100, *_ = svds(test_vectors_T, k = 100)
test_vectors_T.shape, test_U_100.shape # размерность - число слов на 100

((92941, 320000), (92941, 100))

In [31]:
# Нормализуем U_100

test_mean_tfidf = np.mean(test_U_100, 0)
test_std_tfidf = np.std(test_U_100, 0)
test_U_100_norm = (test_U_100 - test_mean_tfidf) / test_std_tfidf

In [32]:
# Создадим словарь {токен: его нормализованный вектор tfidf размерности 100}

test_tfidf_dict = dict(zip(vectorizer.get_feature_names(), test_U_100_norm))

In [37]:
test_loader = DataLoader(
    TwitterDataset_TFIDF_plus_W2V(test_data, "text", "emotion", word2vec, test_tfidf_dict),
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

# НС 4 слоя

# {'Test Loss': 0.816218121554524, 'Test Acc': 0.587182383186901} TfidfVectorizer(min_df=3) 

# НС 2 слоя

# {'Test Loss': 0.8619360493394894, 'Test Acc': 0.5926767172523961} TfidfVectorizer(min_df=3)

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.8619360493394894, 'Test Acc': 0.5926767172523961}


Использование tf-idf как добавление в конец к вектора w2v ситуацию не улучшило, качество на тесте упало - 0.59